# Trail Recommendations

List everything that we need to do.

In [1]:
import Trailforks as tf
import TrailforksScraper as tfs
from bs4 import BeautifulSoup
import requests


## Getting data from Trailforks

- 'activitytype=6' for hiking
- Trailstyle = Popularity

Trailforks api documentation: https://www.trailforks.com/about/api/#!/trail

Trailforks popularity heatmap: https://www.trailforks.com/region/united-states/?activitytype=6&z=10.4&lat=48.30699&lon=-120.42421&trailstyle=popularity

Trailforks popularity scores (sorted in descending order): https://www.trailforks.com/region/united-states/trails/?sort=t.popularity_score&order=desc&difficulty=2,3,4,11,9,5,6,8&activitytype=6

In [2]:
trailForksScrapper = tfs.trailforksScrapper()

In [3]:
north_carolina = trailForksScrapper.fetchTrailsByRegionAndPages('north-carolina',23)
washington = trailForksScrapper.fetchTrailsByRegionAndPages('washington',72)
print(len(north_carolina))
print(len(washington))
washington.head()

2300
7200


,Unnamed: 0,title,Unnamed: 2,riding area,rating,distance,descent,climb,popularity_score
0,NaN,unknown,NaN,Swan Creek Park,NaN,49 ft,-4 ft,NaN,0
1,NaN,unknown,NaN,Grand Ridge Park,NaN,442 ft,-32 ft,NaN,0
2,NaN,unknown,NaN,Tolt MacDonald Park,NaN,49 ft,NaN,4 ft,0
3,NaN,Iron Peak #1399,NaN,Teanaway,NaN,3 miles,-519 ft,"2,203 ft",0
4,NaN,Frog Trail,NaN,Pilchuck Tree Farm,NaN,485 ft,-9 ft,52 ft,0


Convert trail titles to have only words separated by -
This will be used in url for scraping checkins.

In [41]:
north_carolina['title'] = north_carolina['title'].str.replace('\W', ' ').str.lower()
north_carolina['title'] = north_carolina['title'].str.replace('[^a-z A-Z]', '').str.strip().str.replace(' ','-')
washington['title'] = washington['title'].str.replace('\W', ' ').str.lower()
washington['title'] = washington['title'].str.replace('[^a-z A-Z]', '').str.strip().str.replace(' ','-')

/var/folders/7v/3vykhk5j10v0p4dt8gphqvyh0000gn/T/ipykernel_30310/17551254.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  north_carolina['title'] = north_carolina['title'].str.replace('\W', '-').str.lower()
/var/folders/7v/3vykhk5j10v0p4dt8gphqvyh0000gn/T/ipykernel_30310/17551254.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  washington['title'] = washington['title'].str.replace('\W', ' ').str.lower()
/var/folders/7v/3vykhk5j10v0p4dt8gphqvyh0000gn/T/ipykernel_30310/17551254.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  washington['title'].str.replace('[^a-z A-Z]', '').str.strip().str.replace(' ','-')


0                                unknown
1                                unknown
2                                unknown
3                              iron-peak
4                             frog-trail
                     ...                
95                           sixth-sense
96    pipeline-watershed-connector-trail
97                           little-maui
98                          lester-trail
99             the-nut--new-uphill-trail
Name: title, Length: 7200, dtype: object

In [3]:
north_carolina.to_csv('north-carolina-trails.csv')
washington.to_csv('washington-trails.csv')

NameError: name 'north_carolina' is not defined

In [4]:
import pandas as pd
#trails_scraped = pd.read_csv('trail-checkins-13-nov.csv')['trail'].unique()

washington = pd.read_csv('washington-trails.csv')
north_carolina = pd.read_csv('north-carolina-trails.csv')
#len(trails_scraped)

In [47]:
trails = washington[~washington['title'].isin(trails_scraped)]['title']
len(trails)

6910

Getting trail stats for all the trails collected previously.

In [5]:
import pandas as pd 

df = pd.DataFrame()
for trail in north_carolina['title']:
    df_trail = trailForksScrapper.fetchTrailStats(trail)
    if df_trail is not None:
        df_trail['trail'] = trail
        df = pd.concat([df,df_trail])

df

/Users/iqraimtiaz/Documents/duke/Courses/510-SDA/TrailRecommendor/TrailforksScraper.py:137: FutureWarning: The default value of regex will change from True to False in a future version.
  checkins_per_date['Check-Ins'] = checkins_per_date['Check-Ins'].str.replace('\W', '')
/Users/iqraimtiaz/Documents/duke/Courses/510-SDA/TrailRecommendor/TrailforksScraper.py:140: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Period'] = df['Period'].str.replace('\W', '')


,Period,Check-Ins,trail
0,1am,1,highland-trail
1,9am,1,highland-trail
2,10am,1,highland-trail
3,11am,2,highland-trail
4,12pm,3,highland-trail
...,...,...,...
2867,2022108,0,highland-trail
2868,2022109,0,highland-trail
2869,20221010,0,highland-trail
2870,20221011,0,highland-trail


In [12]:
df

,Hour,Check-Ins,Month,Year,Date,trail
0,1 am,7,NaN,NaN,NaN,frog-trail
1,2 am,1,NaN,NaN,NaN,frog-trail
2,4 am,16,NaN,NaN,NaN,frog-trail
3,5 am,4,NaN,NaN,NaN,frog-trail
4,6 am,1,NaN,NaN,NaN,frog-trail
...,...,...,...,...,...,...
2867,NaN,0,NaN,NaN,(2022 10 8),south-shoreline-trail
2868,NaN,0,NaN,NaN,(2022 10 9),south-shoreline-trail
2869,NaN,0,NaN,NaN,(2022 10 10),south-shoreline-trail
2870,NaN,0,NaN,NaN,(2022 10 11),south-shoreline-trail


In [13]:
import numpy as np
np.add.reduce(df[['Hour','Month','Year','Date']].astype(str), axis=1)

0                           1 amnannannan
1                           2 amnannannan
2                           4 amnannannan
3                           5 amnannannan
4                           6 amnannannan
                      ...                
2867                 nannannan(2022 10 8)
2868                 nannannan(2022 10 9)
2869                nannannan(2022 10 10)
2870                nannannan(2022 10 11)
2871                nannannan(2022 10 12)
Length: 4719374, dtype: object

In [ ]:
#trailForksScrapper.fetchTrailStats('kelley-hollow-road')
checkins = trailForksScrapper.fetchTrailStats('rattlesnake-ledge-trail')

In [ ]:
checkins

,Hour,Check-Ins,Month,Year,Date
0,1 am,6,NaN,NaN,NaN
1,2 am,2,NaN,NaN,NaN
2,3 am,2,NaN,NaN,NaN
3,4 am,9,NaN,NaN,NaN
4,5 am,8,NaN,NaN,NaN
...,...,...,...,...,...
2866,NaN,0,NaN,NaN,(2022 10 7)
2867,NaN,0,NaN,NaN,(2022 10 8)
2868,NaN,0,NaN,NaN,(2022 10 9)
2869,NaN,2,NaN,NaN,(2022 10 10)
